In [287]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import log_loss
%matplotlib inline
pd.set_option('display.max_columns', 500)
print("Seaborn version: ", sns.__version__)

Seaborn version:  0.9.0


In [271]:
teams = pd.read_csv('D1_teams.csv')
teams.drop(columns=['code_ncaa','school_ncaa','turner_name','league_name','league_alias','conf_alias',
                    'conf_id','division_name','division_alias','division_id','venue_id'], inplace=True)
teams.info()
teams.set_index('id',inplace=True)
teams.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 3 columns):
id                351 non-null object
kaggle_team_id    351 non-null int64
conf_name         351 non-null object
dtypes: int64(1), object(2)
memory usage: 8.3+ KB


,kaggle_team_id,conf_name
id,,
fe406882-9f22-495e-9df6-ef357a6803c6,1343,Ivy
ca478771-aa3d-4231-81e0-b70f519134fb,1463,Ivy
5c7bf63f-bc39-43c5-9907-73b50b7a6b34,1217,Ivy
d60357bd-1205-42e9-9092-d986a2843a34,1171,Ivy
88ff8c00-958e-4ccf-a21d-77fab9e93692,1165,Ivy


In [333]:
summary_2015_data = pd.read_csv('Season_2015_summary.csv')
summary_2015_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 18 columns):
market                351 non-null object
team_id               351 non-null object
points_avg            351 non-null float64
opp_pts_avg           351 non-null float64
possesion_avg         351 non-null float64
fg_pct                351 non-null float64
allow_fg_pct          351 non-null float64
off_rebs_avg          351 non-null float64
allow_off_rebs_avg    351 non-null float64
def_rebs_avg          351 non-null float64
allow_def_rebs_avg    351 non-null float64
ft_att_avg            351 non-null float64
allow_ft_att_avg      351 non-null float64
ft_pct                351 non-null float64
turnover_avg          351 non-null float64
take_away_avg         351 non-null float64
win_pct               351 non-null float64
off_rating            351 non-null float64
dtypes: float64(16), object(2)
memory usage: 49.4+ KB


In [273]:
game_data = pd.read_csv('tournament_results.csv')
game_data.drop(columns=['days_from_epoch'],inplace=True)
game_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2117 entries, 0 to 2116
Data columns (total 26 columns):
season                 2117 non-null int64
round                  2117 non-null int64
game_date              2117 non-null object
day                    2117 non-null object
win_seed               2117 non-null int64
win_region             2117 non-null object
win_market             2117 non-null object
win_name               2117 non-null object
win_alias              2117 non-null object
win_team_id            2117 non-null object
win_school_ncaa        2117 non-null object
win_code_ncaa          2117 non-null int64
win_kaggle_team_id     2117 non-null int64
win_pts                2117 non-null int64
lose_seed              2117 non-null int64
lose_region            2117 non-null object
lose_market            2117 non-null object
lose_name              2117 non-null object
lose_alias             2117 non-null object
lose_team_id           2117 non-null object
lose_school_ncaa    

In [274]:
tourn_data_2016 = game_data[game_data['season']==2016].copy()
tourn_data_2016.drop(columns=['day','win_region','win_alias','lose_region','lose_alias','lose_code_ncaa',
                              'win_school_ncaa','win_code_ncaa','num_ot','academic_year'],inplace=True)
tourn_data_2016.shape

(67, 16)

In [275]:
tourn_data_2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67 entries, 36 to 2028
Data columns (total 16 columns):
season                 67 non-null int64
round                  67 non-null int64
game_date              67 non-null object
win_seed               67 non-null int64
win_market             67 non-null object
win_name               67 non-null object
win_team_id            67 non-null object
win_kaggle_team_id     67 non-null int64
win_pts                67 non-null int64
lose_seed              67 non-null int64
lose_market            67 non-null object
lose_name              67 non-null object
lose_team_id           67 non-null object
lose_school_ncaa       67 non-null object
lose_kaggle_team_id    67 non-null int64
lose_pts               67 non-null int64
dtypes: int64(8), object(8)
memory usage: 8.9+ KB


In [276]:
tourn_data_2016.reset_index(drop=True,inplace=True)
tourn_data_2016.head()

,season,round,game_date,win_seed,win_market,win_name,win_team_id,win_kaggle_team_id,win_pts,lose_seed,lose_market,lose_name,lose_team_id,lose_school_ncaa,lose_kaggle_team_id,lose_pts
0,2016,64,2016-03-18,10,Virginia Commonwealth,Rams,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,1433,75,7,Oregon State,Beavers,532d3874-b4b3-4c5c-acc6-749a6db26c8f,Oregon St.,1333,67
1,2016,64,2016-03-17,3,Utah,Utes,0d037a5d-827a-44dd-8b70-57603d671d5d,1428,80,14,Fresno State,Bulldogs,dd8db4d8-d984-4cab-b7f6-22c6b8c2c45f,Fresno St.,1201,69
2,2016,64,2016-03-18,1,Oregon,Ducks,1da70895-f77f-44ef-b216-d63c02e696eb,1332,91,16,Holy Cross,Crusaders,31aedd91-a77e-46c1-8bdc-80e9860c159d,Holy Cross,1221,52
3,2016,16,2016-03-24,1,Oregon,Ducks,1da70895-f77f-44ef-b216-d63c02e696eb,1332,82,4,Duke,Blue Devils,faeb1160-5d15-4f26-99fc-c441cf21fc7f,Duke,1181,68
4,2016,32,2016-03-20,1,Oregon,Ducks,1da70895-f77f-44ef-b216-d63c02e696eb,1332,69,8,Saint Joseph's (PA),Hawks,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,Saint Joseph's,1386,64


In [277]:
columns =['season','round','game_date','game_result',
          'team','team_seed', 'team_id',
          'opp_team','opp_team_seed','opp_team_id' 
         ]
tourn_games = pd.DataFrame(columns=columns)
tourn_games

,season,round,game_date,game_result,team,team_seed,team_id,opp_team,opp_team_seed,opp_team_id


In [278]:
tourn_games = pd.DataFrame(columns=columns,index=range( tourn_data_2016.shape[0]),dtype='int32')


In [279]:
games = tourn_data_2016.drop(columns=['win_pts','win_kaggle_team_id','lose_school_ncaa','lose_kaggle_team_id','lose_pts'])
games['game_result'] = 1
games.game_result = games.game_result.astype(int)
games.rename(columns={"win_seed":"team_seed","win_market":"team","win_team_id":"team_id"}, inplace=True)
games.rename(columns={"lose_seed":"opp_team_seed","lose_market":"opp_team","lose_team_id":"opp_team_id"}, inplace=True)
games


,season,round,game_date,team_seed,team,win_name,team_id,opp_team_seed,opp_team,lose_name,opp_team_id,game_result
0,2016,64,2016-03-18,10,Virginia Commonwealth,Rams,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,7,Oregon State,Beavers,532d3874-b4b3-4c5c-acc6-749a6db26c8f,1
1,2016,64,2016-03-17,3,Utah,Utes,0d037a5d-827a-44dd-8b70-57603d671d5d,14,Fresno State,Bulldogs,dd8db4d8-d984-4cab-b7f6-22c6b8c2c45f,1
2,2016,64,2016-03-18,1,Oregon,Ducks,1da70895-f77f-44ef-b216-d63c02e696eb,16,Holy Cross,Crusaders,31aedd91-a77e-46c1-8bdc-80e9860c159d,1
3,2016,16,2016-03-24,1,Oregon,Ducks,1da70895-f77f-44ef-b216-d63c02e696eb,4,Duke,Blue Devils,faeb1160-5d15-4f26-99fc-c441cf21fc7f,1
4,2016,32,2016-03-20,1,Oregon,Ducks,1da70895-f77f-44ef-b216-d63c02e696eb,8,Saint Joseph's (PA),Hawks,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,1
5,2016,64,2016-03-18,8,Saint Joseph's (PA),Hawks,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,9,Cincinnati,Bearcats,a17dfc54-415f-49c3-a2e6-d6fa9db18b0a,1
6,2016,32,2016-03-20,3,Texas A&M,Aggies,ef184799-d79b-49d9-a662-991eaf4044cd,11,Northern Iowa,Panthers,2c93a75d-5449-4ec1-9ca2-1e6c812399a4,1
7,2016,64,2016-03-18,3,Texas A&M,Aggies,ef184799-d79b-49d9-a662-991eaf4044cd,14,Green Bay,Phoenix,61a3e5ab-1be3-4694-b83f-edae0953f409,1
8,2016,68,2016-03-15,16,Florida Gulf Coast,Eagles,5326c079-3dc3-468f-a601-b776dd9c1e62,16,Fairleigh Dickinson,Knights,b71d5a1b-2671-4e5a-b94b-06bfb22a27dd,1
9,2016,64,2016-03-17,9,Providence,Friars,7d3b9c72-cbdd-4b64-9aa3-a5bde9b75fdc,8,USC,Trojans,3a000455-de7c-4ca8-880e-abdce7f21da9,1


In [330]:
tourn_games = pd.DataFrame(columns=columns,index=range( tourn_data_2016.shape[0]),dtype='int32')
print(tourn_games.shape)
print(games.shape)
tourn_games[['season','round','game_date','game_result','team','team_seed','team_id',
             'opp_team_seed','opp_team','opp_team_id']] = games[['season','round','game_date','game_result','team','team_seed',
                                                                'team_id','opp_team_seed','opp_team','opp_team_id']]

tourn_games.head()

(67, 10)
(67, 12)


,season,round,game_date,game_result,team,team_seed,team_id,opp_team,opp_team_seed,opp_team_id
0,2016,64,2016-03-18,1,Virginia Commonwealth,10,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,Oregon State,7,532d3874-b4b3-4c5c-acc6-749a6db26c8f
1,2016,64,2016-03-17,1,Utah,3,0d037a5d-827a-44dd-8b70-57603d671d5d,Fresno State,14,dd8db4d8-d984-4cab-b7f6-22c6b8c2c45f
2,2016,64,2016-03-18,1,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Holy Cross,16,31aedd91-a77e-46c1-8bdc-80e9860c159d
3,2016,16,2016-03-24,1,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Duke,4,faeb1160-5d15-4f26-99fc-c441cf21fc7f
4,2016,32,2016-03-20,1,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Saint Joseph's (PA),8,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8


In [326]:
tourn_games.shape

(67, 10)

In [331]:
tourn_games['copy_team'] = tourn_games['team']
tourn_games['copy_team_seed'] = tourn_games['team_seed']
tourn_games['copy_team_id'] = tourn_games['team_id']
tourn_games.loc[1::2,'team'] = tourn_games.loc[1::2,'opp_team']
tourn_games.loc[1::2,'opp_team'] = tourn_games.loc[1::2,'copy_team']
tourn_games.loc[1::2,'team_seed'] = tourn_games.loc[1::2,'opp_team_seed']
tourn_games.loc[1::2,'opp_team_seed'] = tourn_games.loc[1::2,'copy_team_seed']
tourn_games.loc[1::2,'team_id'] = tourn_games.loc[1::2,'opp_team_id']
tourn_games.loc[1::2,'opp_team_id'] = tourn_games.loc[1::2,'copy_team_id']
tourn_games.loc[1::2,'game_result'] = 0
tourn_games.drop(columns=['copy_team','copy_team_seed','copy_team_id'],inplace=True)
tourn_games.head()

,season,round,game_date,game_result,team,team_seed,team_id,opp_team,opp_team_seed,opp_team_id
0,2016,64,2016-03-18,1,Virginia Commonwealth,10,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,Oregon State,7,532d3874-b4b3-4c5c-acc6-749a6db26c8f
1,2016,64,2016-03-17,0,Fresno State,14,dd8db4d8-d984-4cab-b7f6-22c6b8c2c45f,Utah,3,0d037a5d-827a-44dd-8b70-57603d671d5d
2,2016,64,2016-03-18,1,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Holy Cross,16,31aedd91-a77e-46c1-8bdc-80e9860c159d
3,2016,16,2016-03-24,0,Duke,4,faeb1160-5d15-4f26-99fc-c441cf21fc7f,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb
4,2016,32,2016-03-20,1,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Saint Joseph's (PA),8,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8


In [334]:
summary_2015_data.set_index('team_id',inplace=True)

In [335]:
summary_2015_data.index
summary_2015_data.info()
tourn_games.info()

tourn_games = tourn_games.join(summary_2015_data,on='team_id', how='left')
tourn_games.head()

<class 'pandas.core.frame.DataFrame'>
Index: 351 entries, b18f34af-a7f1-4659-a2e5-fc11a31cd316 to b47d10b8-a2a5-47df-a2f9-7bd0b9d51beb
Data columns (total 17 columns):
market                351 non-null object
points_avg            351 non-null float64
opp_pts_avg           351 non-null float64
possesion_avg         351 non-null float64
fg_pct                351 non-null float64
allow_fg_pct          351 non-null float64
off_rebs_avg          351 non-null float64
allow_off_rebs_avg    351 non-null float64
def_rebs_avg          351 non-null float64
allow_def_rebs_avg    351 non-null float64
ft_att_avg            351 non-null float64
allow_ft_att_avg      351 non-null float64
ft_pct                351 non-null float64
turnover_avg          351 non-null float64
take_away_avg         351 non-null float64
win_pct               351 non-null float64
off_rating            351 non-null float64
dtypes: float64(16), object(1)
memory usage: 49.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex

,season,round,game_date,game_result,team,team_seed,team_id,opp_team,opp_team_seed,opp_team_id,market,points_avg,opp_pts_avg,possesion_avg,fg_pct,allow_fg_pct,off_rebs_avg,allow_off_rebs_avg,def_rebs_avg,allow_def_rebs_avg,ft_att_avg,allow_ft_att_avg,ft_pct,turnover_avg,take_away_avg,win_pct,off_rating
0,2016,64,2016-03-18,1,Virginia Commonwealth,10,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,Oregon State,7,532d3874-b4b3-4c5c-acc6-749a6db26c8f,Virginia Commonwealth,77.250,67.778,81.511111,0.451296,0.440627,10.667,8.083,23.583,23.139,20.028,20.222,0.692094,11.333,14.722,0.694,94.772355
1,2016,64,2016-03-17,0,Fresno State,14,dd8db4d8-d984-4cab-b7f6-22c6b8c2c45f,Utah,3,0d037a5d-827a-44dd-8b70-57603d671d5d,Fresno State,74.455,71.424,79.993939,0.432080,0.423374,9.697,9.000,23.788,24.939,22.788,25.485,0.698138,10.424,14.606,0.697,93.075232
2,2016,64,2016-03-18,1,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Holy Cross,16,31aedd91-a77e-46c1-8bdc-80e9860c159d,Oregon,78.432,69.027,79.254054,0.463695,0.427294,10.081,9.081,22.757,21.405,23.676,18.378,0.715753,10.973,13.946,0.811,98.963306
3,2016,16,2016-03-24,0,Duke,4,faeb1160-5d15-4f26-99fc-c441cf21fc7f,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Duke,81.111,72.472,78.594444,0.460422,0.444742,10.222,11.028,22.417,22.528,24.056,15.778,0.722864,9.667,11.667,0.694,103.202092
4,2016,32,2016-03-20,1,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Saint Joseph's (PA),8,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,Oregon,78.432,69.027,79.254054,0.463695,0.427294,10.081,9.081,22.757,21.405,23.676,18.378,0.715753,10.973,13.946,0.811,98.963306


In [336]:
tourn_games = tourn_games.join(summary_2015_data,on='opp_team_id',lsuffix='_t',rsuffix='_o', how='left')
tourn_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 44 columns):
season                  67 non-null int64
round                   67 non-null int64
game_date               67 non-null object
game_result             67 non-null int32
team                    67 non-null object
team_seed               67 non-null int64
team_id                 67 non-null object
opp_team                67 non-null object
opp_team_seed           67 non-null int64
opp_team_id             67 non-null object
market_t                67 non-null object
points_avg_t            67 non-null float64
opp_pts_avg_t           67 non-null float64
possesion_avg_t         67 non-null float64
fg_pct_t                67 non-null float64
allow_fg_pct_t          67 non-null float64
off_rebs_avg_t          67 non-null float64
allow_off_rebs_avg_t    67 non-null float64
def_rebs_avg_t          67 non-null float64
allow_def_rebs_avg_t    67 non-null float64
ft_att_avg_t            67 non-nu

In [337]:
tourn_games

,season,round,game_date,game_result,team,team_seed,team_id,opp_team,opp_team_seed,opp_team_id,market_t,points_avg_t,opp_pts_avg_t,possesion_avg_t,fg_pct_t,allow_fg_pct_t,off_rebs_avg_t,allow_off_rebs_avg_t,def_rebs_avg_t,allow_def_rebs_avg_t,ft_att_avg_t,allow_ft_att_avg_t,ft_pct_t,turnover_avg_t,take_away_avg_t,win_pct_t,off_rating_t,market_o,points_avg_o,opp_pts_avg_o,possesion_avg_o,fg_pct_o,allow_fg_pct_o,off_rebs_avg_o,allow_off_rebs_avg_o,def_rebs_avg_o,allow_def_rebs_avg_o,ft_att_avg_o,allow_ft_att_avg_o,ft_pct_o,turnover_avg_o,take_away_avg_o,win_pct_o,off_rating_o
0,2016,64,2016-03-18,1,Virginia Commonwealth,10,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,Oregon State,7,532d3874-b4b3-4c5c-acc6-749a6db26c8f,Virginia Commonwealth,77.250,67.778,81.511111,0.451296,0.440627,10.667,8.083,23.583,23.139,20.028,20.222,0.692094,11.333,14.722,0.694,94.772355,Oregon State,71.839,70.710,77.187096,0.438724,0.431779,8.677,9.742,22.194,23.871,21.194,22.000,0.680365,11.065,13.194,0.581,93.070879
1,2016,64,2016-03-17,0,Fresno State,14,dd8db4d8-d984-4cab-b7f6-22c6b8c2c45f,Utah,3,0d037a5d-827a-44dd-8b70-57603d671d5d,Fresno State,74.455,71.424,79.993939,0.432080,0.423374,9.697,9.000,23.788,24.939,22.788,25.485,0.698138,10.424,14.606,0.697,93.075232,Utah,76.086,69.371,75.577142,0.483704,0.421273,8.171,8.686,25.057,20.457,20.943,14.171,0.724420,11.971,10.229,0.743,100.672917
2,2016,64,2016-03-18,1,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Holy Cross,16,31aedd91-a77e-46c1-8bdc-80e9860c159d,Oregon,78.432,69.027,79.254054,0.463695,0.427294,10.081,9.081,22.757,21.405,23.676,18.378,0.715753,10.973,13.946,0.811,98.963306,Holy Cross,64.771,69.857,72.062857,0.412234,0.458868,6.400,7.971,20.571,24.714,19.371,17.886,0.687316,10.600,12.543,0.429,89.881848
3,2016,16,2016-03-24,0,Duke,4,faeb1160-5d15-4f26-99fc-c441cf21fc7f,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Duke,81.111,72.472,78.594444,0.460422,0.444742,10.222,11.028,22.417,22.528,24.056,15.778,0.722864,9.667,11.667,0.694,103.202092,Oregon,78.432,69.027,79.254054,0.463695,0.427294,10.081,9.081,22.757,21.405,23.676,18.378,0.715753,10.973,13.946,0.811,98.963306
4,2016,32,2016-03-20,1,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Saint Joseph's (PA),8,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,Oregon,78.432,69.027,79.254054,0.463695,0.427294,10.081,9.081,22.757,21.405,23.676,18.378,0.715753,10.973,13.946,0.811,98.963306,Saint Joseph's (PA),77.222,70.056,78.538888,0.453321,0.416592,8.111,8.278,26.028,23.861,22.389,15.417,0.715881,9.778,10.972,0.778,98.323548
5,2016,64,2016-03-18,0,Cincinnati,9,a17dfc54-415f-49c3-a2e6-d6fa9db18b0a,Saint Joseph's (PA),8,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,Cincinnati,73.273,63.333,78.672727,0.429798,0.393288,12.000,9.545,24.152,21.455,19.939,15.606,0.705167,10.697,12.818,0.667,93.136122,Saint Joseph's (PA),77.222,70.056,78.538888,0.453321,0.416592,8.111,8.278,26.028,23.861,22.389,15.417,0.715881,9.778,10.972,0.778,98.323548
6,2016,32,2016-03-20,1,Texas A&M,3,ef184799-d79b-49d9-a662-991eaf4044cd,Northern Iowa,11,2c93a75d-5449-4ec1-9ca2-1e6c812399a4,Texas A&M,76.432,66.432,80.713513,0.448822,0.406988,11.189,9.000,24.919,21.892,22.730,18.514,0.671819,11.946,13.865,0.757,94.695955,Northern Iowa,68.343,64.000,68.011428,0.452736,0.424523,4.229,8.057,23.171,24.286,17.600,13.743,0.758117,9.286,11.343,0.629,100.487313
7,2016,64,2016-03-18,0,Green Bay,14,61a3e5ab-1be3-4694-b83f-edae0953f409,Texas A&M,3,ef184799-d79b-49d9-a662-991eaf4044cd,Green Bay,82.676,80.853,87.947058,0.442534,0.453439,10.941,9.941,23.853,26.294,27.441,22.471,0.655949,11.971,16.265,0.618,94.007090,Texas A&M,76.432,66.432,80.713513,0.448822,0.406988,11.189,9.000,24.919,21.892,22.730,18.514,0.671819,11.946,13.865,0.757,94.695955
8,2016,68,2016-03-15,1,Florida Gulf Coast,16,5326c079-3dc3-468f-a601-b776dd9c1e62,Fairleigh Dickinson,16,b71d5a1b-2671-4e5a-b94b-06bfb22a27dd,Florida Gulf Coast,75.968,72.935,80.832258,0.472267,0.420467,9.613,8.968,26.226,22.839,22.323,21.484,0.641618,12.000,11.032,0.548,93.981960,Fairleigh 

,season,round,game_date,game_result,team,team_seed,team_id,opp_team,opp_team_seed,opp_team_id,market_t,points_avg_t,opp_pts_avg_t,possesion_avg_t,fg_pct_t,allow_fg_pct_t,off_rebs_avg_t,allow_off_rebs_avg_t,def_rebs_avg_t,allow_def_rebs_avg_t,ft_att_avg_t,allow_ft_att_avg_t,ft_pct_t,turnover_avg_t,take_away_avg_t,win_pct_t,off_rating_t,market_o,points_avg_o,opp_pts_avg_o,possesion_avg_o,fg_pct_o,allow_fg_pct_o,off_rebs_avg_o,allow_off_rebs_avg_o,def_rebs_avg_o,allow_def_rebs_avg_o,ft_att_avg_o,allow_ft_att_avg_o,ft_pct_o,turnover_avg_o,take_away_avg_o,win_pct_o,off_rating_o
1,2016,64,2016-03-17,1,Utah,3,0d037a5d-827a-44dd-8b70-57603d671d5d,Fresno State,14,dd8db4d8-d984-4cab-b7f6-22c6b8c2c45f,Utah,76.086,69.371,75.577142,0.483704,0.421273,8.171,8.686,25.057,20.457,20.943,14.171,0.724420,11.971,10.229,0.743,100.672917,Fresno State,74.455,71.424,79.993939,0.432080,0.423374,9.697,9.000,23.788,24.939,22.788,25.485,0.698138,10.424,14.606,0.697,93.075232
3,2016,16,2016-03-24,1,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Duke,4,faeb1160-5d15-4f26-99fc-c441cf21fc7f,Oregon,78.432,69.027,79.254054,0.463695,0.427294,10.081,9.081,22.757,21.405,23.676,18.378,0.715753,10.973,13.946,0.811,98.963306,Duke,81.111,72.472,78.594444,0.460422,0.444742,10.222,11.028,22.417,22.528,24.056,15.778,0.722864,9.667,11.667,0.694,103.202092
5,2016,64,2016-03-18,1,Saint Joseph's (PA),8,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,Cincinnati,9,a17dfc54-415f-49c3-a2e6-d6fa9db18b0a,Saint Joseph's (PA),77.222,70.056,78.538888,0.453321,0.416592,8.111,8.278,26.028,23.861,22.389,15.417,0.715881,9.778,10.972,0.778,98.323548,Cincinnati,73.273,63.333,78.672727,0.429798,0.393288,12.000,9.545,24.152,21.455,19.939,15.606,0.705167,10.697,12.818,0.667,93.136122
7,2016,64,2016-03-18,1,Texas A&M,3,ef184799-d79b-49d9-a662-991eaf4044cd,Green Bay,14,61a3e5ab-1be3-4694-b83f-edae0953f409,Texas A&M,76.432,66.432,80.713513,0.448822,0.406988,11.189,9.000,24.919,21.892,22.730,18.514,0.671819,11.946,13.865,0.757,94.695955,Green Bay,82.676,80.853,87.947058,0.442534,0.453439,10.941,9.941,23.853,26.294,27.441,22.471,0.655949,11.971,16.265,0.618,94.007090
9,2016,64,2016-03-17,1,Providence,9,7d3b9c72-cbdd-4b64-9aa3-a5bde9b75fdc,USC,8,3a000455-de7c-4ca8-880e-abdce7f21da9,Providence,73.629,70.143,79.840000,0.420927,0.439768,9.714,8.629,23.029,24.543,22.314,16.514,0.724712,11.114,14.086,0.686,92.220154,USC,80.500,74.618,83.676470,0.459523,0.417156,9.882,11.029,26.029,23.324,22.059,18.441,0.676000,12.000,11.824,0.618,96.203867
11,2016,16,2016-03-25,1,Syracuse,10,8cd24de1-2be8-4b77-bcfa-47be0495a5f4,Gonzaga,11,2f4d21f8-6d5f-48a5-abca-52a30583871a,Syracuse,69.973,65.108,75.713513,0.424592,0.412124,10.378,10.973,21.946,22.243,19.892,16.081,0.687500,11.486,12.919,0.622,92.418076,Gonzaga,78.686,66.000,76.845714,0.484638,0.398012,8.829,8.657,27.171,20.543,19.829,16.971,0.757925,11.257,10.029,0.771,102.394408
13,2016,8,2016-03-27,1,Syracuse,10,8cd24de1-2be8-4b77-bcfa-47be0495a5f4,Virginia,1,56913910-87f7-4ad7-ae3b-5cd9fb218fd9,Syracuse,69.973,65.108,75.713513,0.424592,0.412124,10.378,10.973,21.946,22.243,19.892,16.081,0.687500,11.486,12.919,0.622,92.418076,Virginia,70.973,60.054,68.972972,0.491085,0.420227,7.595,6.838,21.703,19.108,17.027,16.676,0.752381,9.108,11.216,0.784,102.899686
15,2016,32,2016-03-20,1,Wisconsin,7,c7569eae-5b93-4197-b204-6f3a62146b25,Xavier,2,0d8a328f-20ce-410b-bed1-15fc01308aaf,Wisconsin,67.800,63.800,73.605714,0.425443,0.426423,9.571,7.800,22.257,21.800,20.800,17.886,0.703297,10.486,11.686,0.629,92.112413,Xavier,80.500,70.353,82.876470,0.451801,0.414925,10.882,8.471,26.441,22.294,25.206,19.853,0.730455,12.382,13.735,0.824,97.132515
17,2016,16,2016-03-24,1,Oklahoma,2,b2fda957-e15c-4fb2-8a13-6e58496f561e,Texas A&M,3,ef184799-d79b-49d9-a662-991eaf4044cd,Oklahoma,79.676,71.054,80.827027,0.457528,0.411663,9.649,10.081,25.784,22.622,19.838,17.730,0.722071,12.757,12.297,0.784,98.575535,Texas A&M,76.432,66.432,80.713513,0.448822,0.406988,11.189,9.000,24.919,21.892,22.730,18.514,0.671819,11.946

In [338]:
numeric_feature_to_scale = ['points_avg_t','opp_pts_avg_t',
                            'off_rebs_avg_t','allow_off_rebs_avg_t',
                            'def_rebs_avg_t','allow_def_rebs_avg_t',
                            'ft_att_avg_t','allow_ft_att_avg_t',
                            'turnover_avg_t','take_away_avg_t',
                            'off_rating_t',
                            'points_avg_o','opp_pts_avg_o',
                            'off_rebs_avg_o','allow_off_rebs_avg_o',
                            'def_rebs_avg_o','allow_def_rebs_avg_o',
                            'ft_att_avg_o','allow_ft_att_avg_o',
                            'turnover_avg_o','take_away_avg_o',
                            'off_rating_o']

game_data_scale = tourn_games[numeric_feature_to_scale].copy()
scaler =StandardScaler()
scaled_features = scaler.fit_transform(game_data_scale.values)
scaled_df = pd.DataFrame(scaled_features, index=game_data_scale.index, columns=game_data_scale.columns)

# Add the categorical columns and previously scaled numerics
scaled_df['team_seed'] = tourn_games['team_seed']
scaled_df['opp_team_seed'] = tourn_games['opp_team_seed']
scaled_df['fg_pct_t'] = tourn_games['fg_pct_t']
scaled_df['allow_fg_pct_t'] = tourn_games['allow_fg_pct_t']
scaled_df['ft_pct_t'] = tourn_games['ft_pct_t']
scaled_df['win_pct_t'] = tourn_games['win_pct_t']

scaled_df['fg_pct_o'] = tourn_games['fg_pct_o']
scaled_df['allow_fg_pct_o'] = tourn_games['allow_fg_pct_o']
scaled_df['ft_pct_o'] = tourn_games['ft_pct_o']
scaled_df['win_pct_o'] = tourn_games['win_pct_o']

scaled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 32 columns):
points_avg_t            67 non-null float64
opp_pts_avg_t           67 non-null float64
off_rebs_avg_t          67 non-null float64
allow_off_rebs_avg_t    67 non-null float64
def_rebs_avg_t          67 non-null float64
allow_def_rebs_avg_t    67 non-null float64
ft_att_avg_t            67 non-null float64
allow_ft_att_avg_t      67 non-null float64
turnover_avg_t          67 non-null float64
take_away_avg_t         67 non-null float64
off_rating_t            67 non-null float64
points_avg_o            67 non-null float64
opp_pts_avg_o           67 non-null float64
off_rebs_avg_o          67 non-null float64
allow_off_rebs_avg_o    67 non-null float64
def_rebs_avg_o          67 non-null float64
allow_def_rebs_avg_o    67 non-null float64
ft_att_avg_o            67 non-null float64
allow_ft_att_avg_o      67 non-null float64
turnover_avg_o          67 non-null float64
take_away_avg_o  

In [339]:
X= scaled_df
y= tourn_games['game_result']


In [341]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 10)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
X_train.head()

(53, 32)
(53,)
(14, 32)
(14,)


,points_avg_t,opp_pts_avg_t,off_rebs_avg_t,allow_off_rebs_avg_t,def_rebs_avg_t,allow_def_rebs_avg_t,ft_att_avg_t,allow_ft_att_avg_t,turnover_avg_t,take_away_avg_t,off_rating_t,points_avg_o,opp_pts_avg_o,off_rebs_avg_o,allow_off_rebs_avg_o,def_rebs_avg_o,allow_def_rebs_avg_o,ft_att_avg_o,allow_ft_att_avg_o,turnover_avg_o,take_away_avg_o,off_rating_o,team_seed,opp_team_seed,fg_pct_t,allow_fg_pct_t,ft_pct_t,win_pct_t,fg_pct_o,allow_fg_pct_o,ft_pct_o,win_pct_o
23,-1.108465,0.430302,-0.554253,0.422125,-0.858139,-0.322745,0.791443,0.083693,0.161801,-0.193693,-0.685104,1.046911,0.828184,-0.139248,-1.193416,-0.590666,-0.088474,1.015441,0.579963,-0.927626,0.290762,0.831288,8,9,0.447548,0.428337,0.742240,0.594,0.466227,0.435338,0.731266,0.667
52,1.551786,0.465553,2.165343,0.219213,0.610363,-1.460193,-0.203152,-0.159540,-0.521157,-0.056546,0.383886,-0.108743,0.704322,-0.082371,1.029304,-0.765674,-0.712238,-0.952370,-1.110285,-1.253113,-2.025390,0.995297,1,6,0.482234,0.416597,0.746973,0.825,0.476004,0.431755,0.742176,0.667
7,1.527273,3.010273,0.935624,0.850969,-0.260913,2.731525,2.492097,1.376927,0.623751,2.077468,-1.019734,0.207458,-0.445900,1.042986,0.260717,0.472011,-0.011861,0.719207,-0.066637,0.591316,1.058191,-0.668668,14,3,0.442534,0.453439,0.655949,0.618,0.448822,0.406988,0.671819,0.757
56,-0.316629,-0.151355,-0.882804,-0.267606,-0.461566,-0.818495,0.355266,-0.668278,-0.539358,-0.794947,0.967985,0.216793,2.213383,0.574619,0.125084,0.414877,1.193358,1.538447,1.388039,1.896388,0.233915,-1.254194,3,14,0.478628,0.430777,0.748705,0.771,0.432954,0.436551,0.709869,0.559
27,-1.566284,-1.584229,-1.178500,-0.648706,-0.729697,0.552351,-1.483505,0.460562,-1.530861,0.419081,-0.140767,1.485428,1.928252,-1.039998,1.262740,0.692725,0.621622,-1.995523,-0.945315,0.009029,-0.393477,1.289249,12,4,0.451613,0.397428,0.728696,0.848,0.503474,0.438222,0.701599,0.657


In [342]:
logreg = LogisticRegression()

result = logreg.fit(X_train,y_train)

print("Coeffs ",logreg.coef_)
print("Intercept ", logreg.intercept_)

Coeffs  [[ 0.27276651  0.18366358  0.1454022   0.07112466 -0.41364485 -0.01658964
  -0.27671419 -0.66914565 -0.39482813  0.45911717  0.22023503 -0.21387798
   0.78440682  0.03447455 -0.35025398 -0.15186638  0.01620058  0.14848201
   1.01981913  0.54152712 -0.66748064 -0.62100707 -0.13499422  0.03153091
   0.07453539  0.08882981  0.13684706  0.24332322  0.07509463  0.08480468
   0.24488374 -0.02243811]]
Intercept  [0.1788168]


In [343]:
y_pred = logreg.predict(X_test)
print("LR model accuracy is %2.2f" % metrics.accuracy_score(y_test, y_pred))

LR model accuracy is 0.71


In [344]:
print("Log loss= ",log_loss(y_test,logreg.predict_proba(X_test)))

Log loss=  0.598504015432347
